<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion2WordsAndMWEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terminology extraction experiments

## Improving and running terminology extraction script on different corpora



In [1]:
# ArmenianWP
# !wget https://heibox.uni-heidelberg.de/f/206d85a0270943e4b87b/?dl=1
# renaming file
# !mv index.html?dl=1 WPhy_vert.txt

# Constitution Republic of Armenia
# !wget https://heibox.uni-heidelberg.de/f/bf54977b17604ec592cd/?dl=1
# renaming file
# !mv index.html?dl=1 Const_RA_l.txt

# Grundgesetz
# !wget https://heibox.uni-heidelberg.de/f/d6c5b31edc84422d9e14/?dl=1
# renaming file
# !mv index.html?dl=1 GG_l.txt

# Origin of the species
# !wget https://heibox.uni-heidelberg.de/f/befc6dbe718b4a37ba74/?dl=1
# renaming file
# !mv index.html?dl=1 OOOS_l.txt

# Europarl DE
# !wget https://heibox.uni-heidelberg.de/f/3ba6122e744e4b7f9c14/?dl=1
# renaming file
# !mv index.html?dl=1 EP_DE_l.txt

# German legal corpus
!wget https://heibox.uni-heidelberg.de/f/fd96c36723b741d4a972/?dl=1
# renaming file
!mv index.html?dl=1 BGH-utf8-lem.zip


--2021-10-22 13:50:06--  https://heibox.uni-heidelberg.de/f/fd96c36723b741d4a972/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/6ea33bf2-3c70-43ef-97b2-db944a36ad34/output-utf8-lem.zip [following]
--2021-10-22 13:50:07--  https://heibox.uni-heidelberg.de/seafhttp/files/6ea33bf2-3c70-43ef-97b2-db944a36ad34/output-utf8-lem.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 673144348 (642M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 641.96M  16.6MB/s    in 43s     

2021-10-22 13:50:51 (14.8 MB/s) - ‘index.html?dl=1’ saved [673144348/673144348]



In [2]:
!unzip BGH-utf8-lem.zip
!rm BGH-utf8-lem.zip

Archive:  BGH-utf8-lem.zip
  inflating: output-utf8-lem.txt     


In [3]:
!mv output-utf8-lem.txt BGHlem.txt

In [5]:
!head --lines=100000 BGHlem.txt >BGHlem1k.txt

In [25]:
# debugging and explaining the algorithm for Stage1:
LLCandidates = []
LWords = ['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']
for klen in range(len(LWords)): # lengths of candidate lists
    klength = klen+1 # true length: for 0 it is le = 1
    print(f'klen:{klength};')
    for i in range(len(LWords) - klen): # positions where candidates start
        print(f'i:{i};')
        LCandidate = LWords[i:i+klength]
        LLCandidates.append(LCandidate)
        print(LCandidate)

        # print(LWords[i])

for LEl in LLCandidates:
    # for el in LEl:
    print(LEl)

klen:1;
i:0;
['Revisionsverfahren']
i:1;
['entstanden']
i:2;
['notwendig']
i:3;
['Auslage']
klen:2;
i:0;
['Revisionsverfahren', 'entstanden']
i:1;
['entstanden', 'notwendig']
i:2;
['notwendig', 'Auslage']
klen:3;
i:0;
['Revisionsverfahren', 'entstanden', 'notwendig']
i:1;
['entstanden', 'notwendig', 'Auslage']
klen:4;
i:0;
['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']
['Revisionsverfahren']
['entstanden']
['notwendig']
['Auslage']
['Revisionsverfahren', 'entstanden']
['entstanden', 'notwendig']
['notwendig', 'Auslage']
['Revisionsverfahren', 'entstanden', 'notwendig']
['entstanden', 'notwendig', 'Auslage']
['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']


In [32]:
#Terminologieextraktion
import os, re, sys
class clProcCorpus(object):
    ''' we will read a text file and return a dictionary
    this will be done on the line by line basis
    The dictionary can be sorted later...
    '''
    # this is a class for processing a corpus

    def __init__(self, FileIN):
        self.DictFrq = {}
        self.processCorpus(FileIN)

    def processCorpus(self, FileIN):
        # here we consider a larger MWE, which has been collected, e.g., 
        # Wahl Schluckebier Nachschlagewerk;
        # graphical user interface
        # in the default version only the longest string is preserved;
        # we try to split it into meaningful smaller units and preserve them as well:
        # Wahl Schluckebier
        # graphical user ; user interface ; interface
        # PoS restrictions apply: 
        # - Adj cannot be at the end; function words (Prepositions, articles, etc.) cannot be on either edge.
        # so we keep the list of tuples: and extract smaller MWEs from the larger strings using PoS restrictions


        INGram = 2 ## -- homework: n-gram size
        # collecting the longest string
        LLTerm = []
        for Line in FileIN:
            Line = Line.strip()
            LLine = re.split('\t', Line)
            
            try:
                Word = LLine[0]
                PoS = LLine[1]
                Lemma = LLine[2]
            except:
                Word = ""
                PoS = ""
                Lemma = ""
            
            #Select the Tags for your langauge
            #if re.match('N.*', PoS) or re.match('A.*', PoS): #Arm
            #if re.match('N.*', PoS) or re.match('J.*', PoS): #EN
            if re.match('N.*', PoS) or re.match('ADJ.*', PoS): #DE
                LLTerm.append(LLine) # all the field as a list, to form the list of lists

                #Terms as Words or Lemmas
                # to compare with the gold standard do we need words, or do we need to generate words from lemas?
                # LTerm.append(Lemma)
                # here we keep all the fields, because sub-sections of the longer MWE would also be analysed using PoS codes

            else: # end of the 'candidate collection'
                ### changed to an algorithm based on the data format: list of lists

                # stage 1: we generate candidate sub-n-grams, starting from the largest one
                L3Candidates = [] # this is the list of all MWE candidates
                IMaxMWE = len(LLTerm) # this is the longest MWE we can get from LLTerm
                for klen in range(IMaxMWE): # lengths of candidate lists
                    klength = klen+1 # true length: for 0 it is le = 1
                    # print(f'klen:{klength};')
                    for i in range(IMaxMWE - klen): # positions where candidates start
                        # print(f'i:{i};')
                        L2Candidate = LLTerm[i:i+klength]
                        L3Candidates.append(L2Candidate)

                # stage 2: filtering by part-of-speech configurations what is impossible
                L3CandidatesFilt = []
                for L2Candidate in L3Candidates: # for each candidate configuration
                    # if Adj at the end -- remove
                    FileDebug.write(str(L2Candidate) + '\n') 
                FileDebug.write('-\n')



                ####
                # -- homework goes here -- 
                # -- break longer MWE into MWEs of size INGram
                '''
                for i in range(len(LTerm) - INGram):
                    # if i+i+INGram > len(LTerm): break
                    try: LWordsMWE = LTerm[i:i+INGram]
                    except: LWordsMWE = []
                    if LWordsMWE:
                        SWordsMWE = ' '.join(LWordsMWE)
                        # testing effectiveness:
                        # SWordsMWE += ' ~~'
                        try:
                            self.DictFrq[SWordsMWE] += 1
                        except:
                            self.DictFrq[SWordsMWE] = 1                        
                '''
                # avoiding adding repeated sequences into dictionary:

                # -- end: homework
                # if we still need longer MWEs: (if not , just put ''' ''' around this block up to self.DictFrq[STerm] = 1)
                # if len(LTerm) != INGram:  

                ## -- this is going to be replaced; correct field identified...             
                ## STerm = ' '.join(LTerm)
                LLTerm = []

                try:
                    # self.DictFrq[STerm] += 1
                    self.DictFrq[str(LLTerm)] += 1
                except:
                    # self.DictFrq[STerm] = 1
                    self.DictFrq[str(LLTerm)] = 1       
                
        return



In [33]:
FileDebug = open('BGH_debug.txt', 'w')
# FileIN = open('BGHlem.txt', 'r')
FileIN = open('BGHlem1k.txt', 'r')

FileOut1w = open('BGH_term1w.txt', 'w')
FileOutMWE = open('BGH_termMWE.txt', 'w')

In [34]:
# save the frequency dictionary into file, by decreasing frequencies
# FileOutput.write( str( DictionaryFrq ) + '\n' )

OCorpus = clProcCorpus(FileIN)
DictionaryFrq = OCorpus.DictFrq


for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=True):
    if re.search(' ', Word):
        FileOutMWE.write(Word + '\t' + str(Frq) + '\n')
    else:
        FileOut1w.write(Word + '\t' + str(Frq) + '\n')